Vamos fazer a mesma coisa que foi feito em enem.ipynb porem com melhorias substanciais, pois foi tido um resultado ruim se levado em conta o potencial.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import QuantileTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Carregar dados
df = pd.read_csv("../../Data/dados_enem_2022_limpo.csv")

# Remover colunas de notas (já temos NOTA_MEDIA)
df = df.drop(['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO'], axis=1)

# Verificar valores da coluna TP_ESCOLA
print("Valores únicos em TP_ESCOLA:", df['TP_ESCOLA'].unique())

# Criar colunas dummy para tipo de escola
df['ESCOLA_PRIVADA'] = (df['TP_ESCOLA'] == 'Privada').astype(int)
df['ESCOLA_PUBLICA'] = (df['TP_ESCOLA'] == 'Publica').astype(int)

# Criar coluna REGIAO_ESC apenas se SG_UF_ESC existir
if 'SG_UF_ESC' in df.columns:
    uf_para_regiao = {
        # Norte
        'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 
        'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
        # Nordeste
        'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 
        'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
        # Centro-Oeste
        'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
        # Sudeste
        'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
        # Sul
        'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
    }
    df['REGIAO_ESC'] = df['SG_UF_ESC'].map(uf_para_regiao)

# Transformação de features numéricas
df['NOTA_MEDIA'] = df['NOTA_MEDIA'] / 1000
idade_max = df['NU_IDADE'].max()
idade = 100
if idade_max > 100:
    print("Erro: Idade maxima {} passa do limite {} para atributo idade!".format(idade_max, idade))
df['NU_IDADE'] = df['NU_IDADE'] / idade

# Verificar colunas disponíveis antes do ColumnTransformer
print("\nColunas disponíveis no DataFrame:", df.columns.tolist())

# Codificação de features categóricas
# Verificar quais colunas categóricas realmente existem e são relevantes
available_categorical = []
for col in ['TP_COR_RACA', 'TP_ESTADO_CIVIL', 'TP_SEXO', 'TP_NACIONALIDADE', 'TP_LINGUA', 'REGIAO_ESC', 'TP_NACIONALIDADE']:
    if col in df.columns:
        available_categorical.append(col)
    else:
        print(f"Aviso: Coluna {col} não encontrada no DataFrame")

features_dropadas = ['NU_INSCRICAO', 'TP_ESCOLA', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC']
df = df.drop(features_dropadas, axis=1)

print("\nColunas categóricas que serão usadas:", available_categorical)

# Identificar colunas numéricas que devem ser mantidas
numeric_features = ['ESCOLA_PRIVADA', 'ESCOLA_PUBLICA', 'NU_IDADE', 'NOTA_MEDIA']
numeric_features = [col for col in numeric_features if col in df.columns]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), available_categorical)
    ],
    remainder='passthrough'
)

# Transformar features categoricas em numéricas
df['NUM_SEXO'] = df['TP_SEXO'].map({'Feminino': 0, 'Masculino': 1})

regioes = ['Norte', 'Nordeste', 'Centro-Oeste', 'Sudeste', 'Sul']
for regiao in regioes:
    df[f'BIN_REGIAO_{regiao}'] = (df['REGIAO_ESC'] == regiao).astype(int)

estados_civis = df['TP_ESTADO_CIVIL'].unique()
for estado in estados_civis:
    df[f'ESTADO_CIVIL_{estado}'] = (df['TP_ESTADO_CIVIL'] == estado).astype(int)

df['LINGUA_INGLES'] = (df['TP_LINGUA'] == 'Inglês').astype(int)

nacionalidades = df['TP_NACIONALIDADE'].unique()
for nac in nacionalidades:
    if (nac == 'Brasileiro(a)'):
        df[f'BRASILEIRO'] = (df['TP_NACIONALIDADE'] == nac).astype(int)
    else:
        df[f'N_BRASILEIRO'] = (df['TP_NACIONALIDADE'] == nac).astype(int)

# Dropar features a nao mais serem utilizadas
features_dropadas = ['TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_NACIONALIDADE', 'TP_LINGUA', 'TP_COR_RACA']
df = df.drop(features_dropadas, axis=1)

# Separar features e target
X = df.drop('NOTA_MEDIA', axis=1)
y = df['NOTA_MEDIA'].values

print("As seguinte features serao utilizadas: {}".format(df.columns.tolist()))

Valores únicos em TP_ESCOLA: ['Publica' 'Privada']

Colunas disponíveis no DataFrame: ['NU_INSCRICAO', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ESCOLA', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'TP_LINGUA', 'NU_IDADE', 'NOTA_MEDIA', 'ESCOLA_PRIVADA', 'ESCOLA_PUBLICA', 'REGIAO_ESC']

Colunas categóricas que serão usadas: ['TP_COR_RACA', 'TP_ESTADO_CIVIL', 'TP_SEXO', 'TP_NACIONALIDADE', 'TP_LINGUA', 'REGIAO_ESC', 'TP_NACIONALIDADE']

Erro durante o pré-processamento:
A given column is not a column of the dataframe

Verifique os tipos de dados das colunas:
TP_COR_RACA                     object
NU_IDADE                       float64
ESCOLA_PRIVADA                   int64
ESCOLA_PUBLICA                   int64
REGIAO_ESC                      object
NUM_SEXO                       float64
BIN_REGIAO_Norte                 int64
BIN_REGIAO_Nordeste              int64
BIN_REGIAO_Centro-Oeste          int64
BIN_REGIAO_Sudeste               int64
BIN_REGIAO_Sul                

In [32]:
df

,TP_COR_RACA,NU_IDADE,NOTA_MEDIA,ESCOLA_PRIVADA,ESCOLA_PUBLICA,REGIAO_ESC,NUM_SEXO,BIN_REGIAO_Norte,BIN_REGIAO_Nordeste,BIN_REGIAO_Centro-Oeste,BIN_REGIAO_Sudeste,BIN_REGIAO_Sul,ESTADO_CIVIL_Solteiro(a),ESTADO_CIVIL_Nao informado,ESTADO_CIVIL_Divordicado(a),ESTADO_CIVIL_Casado(a),ESTADO_CIVIL_Viuvo(a),LINGUA_INGLES,BRASILEIRO,N_BRASILEIRO
0,Parda,0.20,0.47194,0,1,Nordeste,NaN,0,1,0,0,0,1,0,0,0,0,0,1,0
1,Branca,0.18,0.79354,0,1,Sudeste,NaN,0,0,0,1,0,1,0,0,0,0,0,1,0
2,Parda,0.17,0.57924,0,1,Centro-Oeste,NaN,0,0,1,0,0,1,0,0,0,0,0,1,0
3,Parda,0.18,0.46996,0,1,Norte,NaN,1,0,0,0,0,1,0,0,0,0,0,1,0
4,Branca,0.18,0.48966,0,1,Sudeste,NaN,0,0,0,1,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191308,Parda,0.18,0.60774,0,1,Nordeste,NaN,0,1,0,0,0,1,0,0,0,0,0,1,0
191309,Parda,0.17,0.49778,0,1,Nordeste,NaN,0,1,0,0,0,1,0,0,0,0,0,1,0
191310,Preta,0.18,0.54680,0,1,Nordeste,NaN,0,1,0,0,0,1,0,0,0,0,0,1,0
191311,Parda,0.18,0.47994,0,1,Nordeste,NaN,0,1,0,0,0,1,0,0,0,0,0,1,0


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class EnhancedENEMRegressor(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.4),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            
            nn.Linear(64, 32),
            nn.ELU(),
            
            nn.Linear(32, 1)
        )
        
        # Inicialização cuidadosa
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu')
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        return self.net(x)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter

# Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Converter para tensores
X_train_tensor = torch.FloatTensor(X_train.toarray() if hasattr(X_train, 'toarray') else X_train)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)
X_test_tensor = torch.FloatTensor(X_test.toarray() if hasattr(X_test, 'toarray') else X_test)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1)

# Criar DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Configuração do treino
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnhancedENEMRegressor(X_train.shape[1]).to(device)

criterion = nn.HuberLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.001)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, 
                                        steps_per_epoch=len(train_loader), 
                                        epochs=50)

# Monitoramento com TensorBoard
writer = SummaryWriter()

# Loop de treinamento
for epoch in range(50):
    model.train()
    train_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        train_loss += loss.item()
    
    # Validação
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    # Métricas
    train_loss /= len(train_loader)
    val_loss /= len(test_loader)
    r2 = r2_score(np.concatenate(all_labels), np.concatenate(all_preds))
    
    # Log no TensorBoard
    writer.add_scalars('Loss', {'train': train_loss, 'val': val_loss}, epoch)
    writer.add_scalar('R2', r2, epoch)
    
    print(f'Epoch {epoch+1:02d} | '
          f'Train Loss: {train_loss:.4f} | '
          f'Val Loss: {val_loss:.4f} | '
          f'R²: {r2:.4f} | '
          f'LR: {scheduler.get_last_lr()[0]:.2e}')

writer.close()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Função para plotar resultados
def plot_results(model, X_test_tensor, y_test_tensor):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor.to(device)).cpu().numpy()
        actuals = y_test_tensor.numpy()
    
    plt.figure(figsize=(12, 5))
    
    # Gráfico de dispersão
    plt.subplot(1, 2, 1)
    plt.scatter(actuals, predictions, alpha=0.3)
    plt.plot([actuals.min(), actuals.max()], [actuals.min(), actuals.max()], 'r--')
    plt.xlabel('Valores Reais')
    plt.ylabel('Predições')
    plt.title(f'R²: {r2_score(actuals, predictions):.4f}')
    
    # Distribuição de erros
    plt.subplot(1, 2, 2)
    errors = predictions.flatten() - actuals.flatten()
    plt.hist(errors, bins=50)
    plt.xlabel('Erros de Predição')
    plt.ylabel('Frequência')
    plt.title(f'MSE: {mean_squared_error(actuals, predictions):.4f}')
    
    plt.tight_layout()
    plt.show()

plot_results(model, X_test_tensor, y_test_tensor)